# The impact of local interactions on an epidemic

This interactive app explores research by Lydia Wren (an erstwhile undergraduate student at the School of Mathematics and Statistics at Sheffield) and myself, which was published in [*Bulletin of Mathematical Biology*](https://link.springer.com/article/10.1007%2Fs11538-021-00961-w).

We look at a model of an epidemic with a simple Susceptible-Infected-Recovered framework. Standard models often assume the infection spreads 'globally' - that is, you are equally likely to catch the infection off any infected individual. In reality, you are much more likely to catch disease off somebody physically close to you. A simple way of modelling this is to assume the population lives on a square grid. Your contacts could then be *either* global across the grid *or* local with your 4 near-neighbours, or some mix of the two. 

Our model uses a parameter, $L$, to control the degree of local interactions. $L=0$ means all contacts are global, $L=1$ means all contacts are local, and $0<L<1$ means a weighted mix of the two (i.e. $L=0.5$ means half global/half local). 

We present the model in 3 different ways:

1.   The standard differential equation model that does not include any spatial structure. This can be used to compare any choice of $L$ you make with the completely global case.
2.   A differential equation model using a 'pair approximation' to include some spatial structure.
3. 10 runs of a stochastic model that is fully spatially structured.

Using the sliders you can choose different values of $L$ and $R_0$ to see how the dynamics change. When you click 'Run model' you will see time-courses from each of the above models, as well as a print out of the total and peak infected from the approximate spatial model.

## Suggested approach

Choose a value of $R_0$. Look what happens with a very small value for $L$ (0.1, say) and compare it with what happens with a very large value (0.9, perhaps). Then choose a few intermediate values and see how the plots and details change. Each time you produce a new plot, click the animation button below afterwards to visualise what is going on.

In [18]:
#!jupyter nbextension enable --py widgetsnbextension --sys-prefix
#!jupyter serverextension enable voila --sys-prefix
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import ipywidgets as widgets
from ipywidgets import interact_manual

In [24]:
#@title
# Import the necessary libraries
#from IPython.display import HTML
plt.rcParams.update({'font.size': 16})

Lw=widgets.FloatSlider(min=0, max=1, step=0.05, value=0, description='$L$')
R0w=widgets.FloatSlider(min=0.5, max=5, step=0.5, value=2, description='$R_0$')

gridsaves=np.zeros([36,20,20])
num_frames=0

def runspatial():
  N=20                     # Grid size (suggested min. of 20 and max. of 50 - the larger, the longer it takes)
  tot=N*N
  GAMMA=1/14               # Recovery rate
  I0=5                     # Starting no. of infecteds
  REPS=10                  # No. of replicates of IBM to do
  L=Lw.value
  R0=R0w.value
  BETA=R0*GAMMA/tot        # Transmission

  # PAIR APPROXIMATION
  def pairs(t,x,BETAP):
      qis=x[3]/x[0]
      sdot=-BETAP*x[0]*(L*qis+(1-L)*x[1])
      idot=BETAP*x[0]*(L*qis+(1-L)*x[1])-GAMMA*x[1]
      ssdot=-2*BETAP*(L*3/4*qis+(1-L)*x[1])*x[2]
      sidot=-BETAP*(L*(1/4+3/4*qis)+(1-L)*x[1])*x[3]-GAMMA*x[3]+BETAP*(L*3/4*qis+(1-L)*x[1])*x[2]
      srdot=-BETAP*(L*3/4*qis+(1-L)*x[1])*x[4]+GAMMA*x[3]
      iidot=-2*GAMMA*x[5]+2*BETAP*(L*(1/4+3/4*qis)+(1-L)*x[1])*x[3]
      irdot=-GAMMA*x[6]+BETAP*(L*3/4*qis+(1-L)*x[1])*x[4]+GAMMA*x[5]
      return [sdot,idot,ssdot,sidot,srdot,iidot,irdot]
      
  # MEAN-FIELD APPROXIMATION
  def disease(t,x,BETAP):
      sdot=-BETAP*x[0]*x[1]
      idot=BETAP*x[0]*x[1]-GAMMA*x[1]
      return sdot, idot

  # Function to count noumbers in each compartment
  def count_type(sirtype):
      current_type=0;
      for i in range(0,N):
          for j in range(0,N):
              if grid[i,j]==sirtype:
                  current_type=current_type+1
      return current_type

  # Function to find nearest-neighbours
  def local_inf(site_x,site_y):
      localinfs=0
      if site_x==0:
          downx=N-1
      else:
          downx=site_x-1
      if site_y==0:
          downy=N-1
      else:
          downy=site_y-1
      if site_x==N-1:
          upx=0
      else:
          upx=site_x+1
      if site_y==N-1:
          upy=0
      else:
          upy=site_y+1

      if grid[downx,site_y]==1:
          localinfs=localinfs+1
      if grid[upx,site_y]==1:
          localinfs=localinfs+1
      if grid[site_x,downy]==1:
          localinfs=localinfs+1
      if grid[site_x,upy]==1:
          localinfs=localinfs+1
      return localinfs

  # Function to check the current scale
  def findscale():
      S=count_type(0)
      I=count_type(1)
      localinf=0
      for i in range(0,N):
          for j in range(0,N):
              if grid[i,j]==0:
                  localinf=local_inf(i,j)+localinf
      if S>0:
          QIS=localinf/S*tot/4
      else:
          QIS=0
      #Set relative parameter values
      scale=GAMMA*I+BETA*S*(L*QIS+(1-L)*I)  
      return scale

  #saves up to 500 plots for animation
  global gridsaves
  sus,inf,rec,tplot = [],[],[],[]
  global num_frames

  gridsaves=np.zeros([90,20,20])
  num_frames=0


  ibmmax=np.zeros(REPS)
  ibmt=np.zeros(REPS)

  # Multiple runs of model
  for reps in range(0,REPS):
      if reps==0:
          print('Running simulation 1', end=" ")
      else:
        print(', %i' % int(reps+1),end=" ")
      plots=-1
      # Set initial conditions
      init_inf=I0
      grid=np.zeros((N,N))
      for i in range(0,init_inf):
          grid[np.random.randint(0,N-1),np.random.randint(0,N-1)]=1
      tsteps=[0]
      infecteds=[count_type(1)]
      current_t=0

      # Main run
      while current_t<180:

          # Find tau-leap
          scale=findscale()
          dt = -np.log(np.random.rand()) / scale
          current_t=tsteps[-1]

          #saves a grid every 2 days
          for k in range(90):
              if reps==0 and current_t<k*2 and current_t+dt>k*2:
                  gridsaves[num_frames][:][:]=grid[:][:]
                  num_frames +=1
                  sus.append(count_type(0)/(N*N))
                  inf.append(count_type(1)/(N*N))
                  rec.append(count_type(2)/(N*N))
                  tplot.append(dt+current_t)

          # Create randomised list of sites to check
          findx=[i for i in range(N)]
          findy=[i for i in range(N)]
          np.random.shuffle(findx)
          np.random.shuffle(findy) 
          flagged=0   # Used to break out of 2nd loop

          #Find event
          if np.random.rand()<GAMMA*infecteds[-1]/scale: #Event is recovery   
              for tryx in findx:
                  if flagged==1:
                      break
                  for tryy in findy:
                      if grid[tryx,tryy]==1:
                          grid[tryx,tryy]=2
                          flagged=1
                          break
          else: #Event is transmission
              if np.random.rand()>L: #Transmission is global
                  for tryx in findx:
                      if flagged==1:
                          break
                      for tryy in findy:
                          if grid[tryx,tryy]==0:                       
                              grid[tryx,tryy]=1
                              flagged=1
                              break
              else: # Transmission is local
                  for tryx in findx:
                      if flagged==1:
                          break
                      for tryy in findy:
                          if grid[tryx,tryy]==0: 
                              if local_inf(tryx,tryy)>0:
                                  grid[tryx,tryy]=1
                                  flagged=1
                                  break

          # Update time and infection lists
          tsteps.append(dt+current_t)
          infecteds.append(count_type(1))
          if infecteds[-1]==0:
              break

      # Plot latest run
      infected_prop=[inf/tot for inf in infecteds]
      ibmmax[reps]=max(infected_prop)
      for j in range(0,len(tsteps)):
          if infected_prop[j]==ibmmax[reps]:
              ibmt[reps]=tsteps[j]
              break
      if reps==0:
        plt.plot(tsteps,infected_prop,'b:',alpha=0.3,label='Fully spatial sims') 
      else:
        plt.plot(tsteps,infected_prop,'b:',alpha=0.3)   



  ETA=R0*GAMMA # In ODE models total pop=1 so beta needs changing
  ts=np.linspace(0,180,2000)
  xx1=solve_ivp(disease,[ts[0],ts[-1]],[1-I0/tot,I0/tot],t_eval=ts,args=(ETA,)) 
  plt.plot(ts,(xx1.y[1]),'k',label='Non-spatial model')



  ts=np.linspace(0,180,2000)
  xxp=solve_ivp(pairs,[ts[0],ts[-1]],[1-I0/tot,I0/tot,(1-I0/tot)**2,(1-I0/tot)*(I0/tot),0,(I0/tot)**2,0],t_eval=ts,args=(ETA,)) 
  plt.plot(ts,(xxp.y[1]),'r',label='Spatial approx. model')
  plt.xlabel('Days')
  plt.ylabel('Proportion Infected')
  plt.legend()
  plt.ylim(0,0.5)
  plt.xlim(0,180)
  #plt.savefig('l09.png')
  plt.show()

  print('By day 180,' , +int(100-100*xxp.y[0,-1]),'per-cent of the population have been infected, and the peak of the infection was', int(100*np.max(xxp.y[1])), 'per-cent')
  return()

display(Lw)
display(R0w)
my_interact_manual4 = interact_manual.options(manual_name="Run model")
my_interact_manual4(runspatial);

FloatSlider(value=0.0, description='$L$', max=1.0, step=0.05)

FloatSlider(value=2.0, description='$R_0$', max=5.0, min=0.5, step=0.5)

interactive(children=(Button(description='Run model', style=ButtonStyle()), Output()), _dom_classes=('widget-i…

Clicking the button below will animate one of the stochastic runs of the model you have just seen plotted, so you can see how the infection spreads. Compare what happens with low and high values of $L$.

*This can take 10-20 seconds to load.*

In [5]:
#@title
#animations for the lattice for the IBM

import matplotlib.colors as clt
from matplotlib import animation, rc
import matplotlib.patches as patches
from IPython.display import HTML

def make_anim():
    print('Building animation - please wait')
    def animate(i):
        N=20
        ax.clear()
        # make color map
        my_cmap = clt.ListedColormap(['b', 'r', 'y'])
        # set the 'bad' values (nan) to be white and transparent
        my_cmap.set_bad(color='w', alpha=0)
        for x in range(N + 1):
            ax.axhline(x, lw=1, color='k', zorder=5)
            ax.axvline(x, lw=1, color='k', zorder=5)    
        # draw the boxes
        ax.imshow(gridsaves[i], interpolation='none', cmap=my_cmap, extent=[0, N, 0, N], zorder=0)
        #legend
        names = ['Susceptible','Infected','Recovered']
        pat = [ patches.Patch(color=['b', 'r', 'y'][j], label="{l}".format(l=names[j]) ) for j in range(len(names)) ]
        ax.legend(handles=pat, loc=6, bbox_to_anchor=(1.05, 0.5), borderaxespad=0. )
        #title and axis
        ax.set_title('Simulated infections through time and space') 
        ax.axis('off')


    fig, ax = plt.subplots(figsize=(10,5))
    plt.close(fig) #needs to be closed or it just prints a stationairy frame
    #animator - change repeat to false to make video stop at the end
    anim = animation.FuncAnimation(ax.figure, animate,interval=500,frames=int(num_frames),repeat=False)
    #HTML function for Jupyter
    display(HTML(anim.to_html5_video()))
    #anim.save('local0.mp4')

my_interact_manual5 = interact_manual.options(manual_name="Animate model")
my_interact_manual5(make_anim);

interactive(children=(Button(description='Animate model', style=ButtonStyle()), Output()), _dom_classes=('widg…

## What does this tell us?

If you look at time-courses for a high value of $L$ (mainly local interactions), you will see that the epidemic is much flatter and smaller than if you choose a low value of $L$ (mainly global interactions). However, this reduction in epidemic severity does not happen steadily/gradually. Even a value of $L$ of $0.4$ or $0.5$ produces pretty similar epidemics to if there were no local interactions at all. You have to have a pretty high proportion of local interactions, often $L>0.8$, to see a large effect of movement restrictions.

You can see why this is from the animation. When $L$ is large, the majority of infections occur locally. Once an individual has infected all their neighbours, they run out of susceptible contacts and the local epidemic slows down. In contrast, when $L$ is small so that infected individuals frequently have global contacts, the infection can easily spread to a new area of the grid and start a new local epidemic.

Another thing to notice is the variation in the time-courses for the stochastic model runs. This is especially evident at low values of $R_0$ and mid-to-high values of $L$, where the peak and total number infected can vary widely for the exact same system and parameters.

For more details, please read the [publication](https://link.springer.com/article/10.1007%2Fs11538-021-00961-w) by me and Lydia.

*Thanks to Lydia Wren and Natasha Ellison for helping with parts of the model and coding.*